In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc
import time

time_start=time.time()

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])

In [3]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [4]:
train=train[['qid','uid','target']]
test=test[['qid','uid']]
member_info=member_info[['uid','l_topic']]
ques_info=ques_info[['qid','qtitlec']]

member_info.drop_duplicates(['uid'],inplace=True)
ques_info.drop_duplicates(['qid'],inplace=True)

In [5]:
train=pd.merge(train,member_info,on='uid',how='left')
test=pd.merge(test,member_info,on='uid',how='left')

train=pd.merge(train,ques_info,on='qid',how='left')
test=pd.merge(test,ques_info,on='qid',how='left')

In [6]:
len(train)+len(test)

10630845

In [7]:
target=train['target']
train.drop(['target'],axis=1,inplace=True)
all_data=pd.concat([train,test],axis=0)
all_data.shape

(10630845, 4)

In [8]:
all_data.head()

,qid,uid,l_topic,qtitlec
0,Q2166419046,M401693808,"T1727,T5310,T3402,T916,T1506,T26329,T7293,T180...","W11058,W272,W2202,W431,W951,W243,W3828,W3037,W263"
1,Q1550017551,M3392373099,"T42595,T3,T8520,T597,T6485,T6212,T25664,T148,T...","W149,W79,W5210,W22869"
2,Q604029601,M2317670257,"T610,T448,T61,T2801,T9019,T65,T233,T190,T55,T5...","W31489,W9218,W3440,W243"
3,Q2350061229,M1618461867,"T5,T33331,T2274,T31,T245,T516,T309,T1326,T119,...","W973,W64958,W8583,W2269,W628,W565,W2200,W590,W483"
4,Q2443223942,M3544409350,-1,"W554,W28208,W396,W51"


In [9]:
all_data['l_topic']=all_data['l_topic'].astype(str)
all_data['qtitlec']=all_data['qtitlec'].astype(str)
all_data['l_topic']=all_data['l_topic'].apply(lambda x:x.split(','))
all_data['qtitlec']=all_data['qtitlec'].apply(lambda x:x.split(','))
all_data['num_ltopic']=all_data['l_topic'].apply(lambda x:len(x))
all_data['num_qtitlec']=all_data['qtitlec'].apply(lambda x:len(x))

In [10]:
all_data['l_topic']=all_data['l_topic'].apply(lambda x:' '.join(x))
all_data['qtitlec']=all_data['qtitlec'].apply(lambda x:' '.join(x))

In [11]:
from tqdm import tqdm
tqdm.pandas()
import numpy as np
#import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gc
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [12]:
tfidf = CountVectorizer(ngram_range=(1,1))
tf_ltopic = tfidf.fit_transform(all_data['l_topic'])
tfidf = CountVectorizer(ngram_range=(1,1))
tf_qtopic = tfidf.fit_transform(all_data['qtitlec'])

In [13]:
print(tf_ltopic.shape)
print(tf_qtopic.shape)

(10630845, 79097)
(10630845, 130248)


In [14]:
from scipy import sparse
all_sparse=sparse.csc_matrix(sparse.hstack((tf_ltopic, tf_qtopic)))
all_sparse.shape

(10630845, 209345)

In [15]:
train_feature=all_sparse[0:train.shape[0]]
test_feature=all_sparse[train.shape[0]:]

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [16]:
def lrstacking():
    num_class=2
    df_stack=pd.DataFrame()
    n_folds=5
    gc.collect()
    print('LR stacking')
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (trn_idx, val_idx) in enumerate(folds.split(target)):
        print('stack:%d' % ((i + 1) ))
        clf = LogisticRegression(random_state=1017, C=8,solver='sag',verbose=1,n_jobs=-1)
        clf.fit(train_feature[trn_idx], target[trn_idx])
        score_va = clf.predict_proba(train_feature[val_idx])
        score_te = clf.predict_proba(test_feature)
        stack_train[val_idx, :] = score_va
        stack_test[:, :] += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_lr_{}'.format(i)] = stack[:, i]
    print('lr done')
    return df_stack

In [17]:
lrstacking=lrstacking()

LR stacking
stack:1
max_iter reached after 1784 seconds


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 29.8min finished


stack:2
max_iter reached after 1394 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 23.2min finished


stack:3
max_iter reached after 1255 seconds


...

stack:4
max_iter reached after 1311 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 21.8min finished


stack:5
max_iter reached after 1362 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 22.7min finished


lr done


In [18]:
def sgdstacking():
    gc.collect()
    print('sgd stacking')
    df_stack=pd.DataFrame()
    n_folds=5
    num_class=2
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        sgd = SGDClassifier(random_state=1017,
                            verbose=1,
                            loss='log',
                            penalty='elasticnet',
                            shuffle=True,
                            n_jobs=20,
                            l1_ratio=0.0001,
                            alpha=1e-05,
                            class_weight=None)
        gc.collect()
        sgd.fit(train_feature[tr], target[tr])
        score_va = sgd.predict_proba(train_feature[va])
        score_te = sgd.predict_proba(test_feature)
        stack_train[va] = score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_sgd_{}'.format(i)] = stack[:, i]
    print('sgd done')
    return df_stack

In [19]:
sgdstacking=sgdstacking()

sgd stacking
stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


-- Epoch 1
Norm: 30.65, NNZs: 197277, Bias: -1.474468, T: 7591329, Avg. loss: 0.623154
Total training time: 8.78 seconds.
-- Epoch 2
Norm: 28.89, NNZs: 197285, Bias: -1.473191, T: 15182658, Avg. loss: 0.451686
Total training time: 17.21 seconds.
-- Epoch 3
Norm: 28.45, NNZs: 197324, Bias: -1.470077, T: 22773987, Avg. loss: 0.447974
Total training time: 27.06 seconds.
-- Epoch 4
Norm: 28.28, NNZs: 197300, Bias: -1.470500, T: 30365316, Avg. loss: 0.446452
Total training time: 36.76 seconds.
-- Epoch 5
Norm: 28.17, NNZs: 197404, Bias: -1.473007, T: 37956645, Avg. loss: 0.445638
Total training time: 46.77 seconds.
stack:2/5
-- Epoch 1
Norm: 30.58, NNZs: 197315, Bias: -1.465340, T: 7591329, Avg. loss: 0.625475
Total training time: 9.05 seconds.
-- Epoch 2
Norm: 28.87, NNZs: 197258, Bias: -1.474226, T: 15182658, Avg. loss: 0.451726
Total training time: 18.71 seconds.
-- Epoch 3
Norm: 28.45, NNZs: 197282, Bias: -1.465607, T: 22773987, Avg. loss: 0.447994
Total training time: 27.57 seconds.
--

In [20]:
def ridgestacking():
    gc.collect()
    print('Ridge stacking')
    df_stack=pd.DataFrame()
    num_class=2
    n_folds=5
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        ridge = RidgeClassifier(random_state=1017,solver='sag',)
        ridge.fit(train_feature[tr], target[tr])
        score_va = ridge._predict_proba_lr(train_feature[va])
        score_te = ridge._predict_proba_lr(test_feature)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_ridge_{}'.format(i)] = stack[:, i]
    print('ridge done')
    return df_stack

In [21]:
ridgestacking=ridgestacking()

Ridge stacking
stack:1/5
stack:2/5
stack:3/5
stack:4/5
stack:5/5
ridge done


In [22]:
result=pd.concat([lrstacking,sgdstacking,ridgestacking],axis=1)
result.head()

,tfc_lr_0,tfc_lr_1,tfc_sgd_0,tfc_sgd_1,tfc_ridge_0,tfc_ridge_1
0,0.935751,0.064249,0.934477,0.065523,0.703916,0.296084
1,0.844869,0.155131,0.815090,0.184910,0.665241,0.334759
2,0.869858,0.130142,0.887802,0.112198,0.673109,0.326891
3,0.854217,0.145783,0.842922,0.157078,0.665344,0.334656
4,0.697233,0.302767,0.754147,0.245853,0.597356,0.402644


In [23]:
len(result)

10630845

In [24]:
result.columns=['tfc_lr_q', 'tfc_lr_1', 'tfc_sgd_q', 'tfc_sgd_1', 'tfc_ridge_q','tfc_ridge_1']

In [25]:
result[['tfc_lr_q', 'tfc_sgd_q', 'tfc_ridge_q']].to_csv('feature_qidtop_tfidf.csv',index=False)

In [26]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 32560.318981409073
